In [1]:
import os
import requests
import geohash2
import pandas as pd
import math
import warnings

import matplotlib.pyplot as plt
plt.rcParams['timezone'] = 'America/Bogota'
import matplotlib.dates as mdates
import seaborn as sns
# Gloabl seaborn Theme
sns.set_theme(style="whitegrid", palette="pastel")

from io import StringIO
from pandas import DataFrame
from datetime import datetime, timezone, timedelta
from scipy.stats import pearsonr
from scipy.stats._result_classes import PearsonRResult
from scipy.stats import NearConstantInputWarning, ConstantInputWarning
from requests import Response
from datetime import datetime
from dotenv import load_dotenv

# Explicitly providing path to '.env'
from pathlib import Path  # Python 3.6+ only
# Load .env variables
_ = load_dotenv(dotenv_path=f"{Path().resolve().parents[1]}/standalone/.env")

# import import_ipynb
# %run "aqi_epa_pm25.ipynb"
# from aqi_epa_pm25 import PM25_MAX_VALUE, pm25_to_aqi

# with the new api
from importnb import imports
with imports("ipynb"):
    from aqi_epa_pm25 import PM25_MAX_VALUE, pm25_to_aqi

PM2.5: 35.9, AQI: 102
PM2.5: 35.9, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 35.5, Max: 55.4
AQI: 102, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 101, Max: 150


## Utils

In [2]:
def to_timestamp(datetime_iso8601: str) -> int:
    """
    Datetime ISO 8601 Format to Timestamp
    TZ='America/Bogota' -05:00

    :params
    :datetime_iso8601: str, Datetime ISO 8601 Format

    :return: int, Timestamp

    :example
        - to_timestamp('2023-03-17T00:00:00-05:00')
            return: 1679029200000
    """
    return int(datetime.fromisoformat(datetime_iso8601).timestamp() * 1000)

In [3]:
def request_influxdb(sql_query: str) -> Response:
    """
    Request to InfluxDB API REST

    :params
    :sql_query: str, InfluxDB SQL query

    :return: Response, InfluxDB response as CSV text
    """
    endpoint = os.getenv("URL_INFLUXDB_QUERY_ENDPOINT", None)
    database = os.getenv("DB_NAME_INFLUXDB", None)
    parameters = {
        'db': database,
        'q': sql_query,
        'epoch': 'ms',
        'format': 'json'
    }
    # To get response as CSV text
    headers = {'Accept': 'application/json'}
    # GET Request
    return requests.get(endpoint, params=parameters, headers=headers)

In [4]:
def query_tangaras(start_timestamp: int, end_timestamp: int) -> str:
    """
    Get InfluxDB SQL query of all Tangara sensors that have reported data over a period of time.

    :params:
    :start_timestamp: int, timestamp datetime value, ms
    :end_timestamp: int, timestamp datetime value, ms

    :return: str, InfluxDB SQL Query
    """
    # Period DateTime
    period_time = f"time >= {start_timestamp}ms AND time <= {end_timestamp}ms"
    # SQL
    sql_query = "SELECT DISTINCT(geo) AS \"geohash\" "\
                "FROM \"fixed_stations_01\" WHERE "\
                "(\"geo3\" = 'd29') AND "\
                f"{period_time} "\
                "GROUP BY \"name\";"
    return sql_query

In [5]:
def query_measure(mac_tangaras: [str], start_timestamp: int, end_timestamp: int, measure: str='pm25', group_by_time: str='30s') -> str:
    """
    Get InfluxDB SQL query for specific measure (datatype) and for each Tangara sensor identified by MAC address between a period of time.

    :params:
    :mac_tangaras: [str], Tangara sensor MAC address
    :start_timestamp: int, timestamp datetime value, ms
    :end_timestamp: int, timestamp datetime value, ms
    :measure: str, choice ['pm25', 'tmp', 'hum']
    :group_by_time: str, choice ['30s', '1m', '1h']

    :return: str, InfluxDB SQL Query
    """
    # Period DateTime
    period_time = f"time >= {start_timestamp}ms AND time <= {end_timestamp}ms"
    # SQL Datatype by Tangara Sensor
    sql_query = ""
    head = "last(" if group_by_time == '30s' else "mean("
    for mac in mac_tangaras:
        sql_query += f"SELECT {head}\"{measure}\") "\
                    "FROM \"fixed_stations_01\" WHERE "\
                    f"(\"name\" = '{mac}') AND "\
                    f"{period_time} " \
                    f"GROUP BY time({group_by_time}) fill(null); "
    return sql_query[:-2]

In [6]:
def to_be_checked(df_sensor: DataFrame, threshold_data_percent: int=80) -> [bool, int, int]:
    """
    Check if the sensor must be checked, because it has not reported enough data.
    Return [bool, int]: [{Does it to be checked?}, {Total data}, {Total missing data}]

    :params:
    :df_sensor: DataFrame, Data reported by Tangara sensor
    :threshold_data_percent: int, Threshold to check enough data reported

    :return: [bool, int, int], Does it not report enough data?
    """
    # Check missing data
    total = df_sensor.shape[0]
    missing_data_percent = round(df_sensor.isna().sum()[0] * 100 / total)
    data_percent = round(df_sensor.count()[0] * 100 / total)
    # Threshold
    if data_percent < threshold_data_percent:
        # to be checked
        return [False, data_percent, missing_data_percent]
    # OK
    return [True, data_percent, missing_data_percent]

In [7]:
def is_corr_ok(df_reference_sensor: DataFrame, df_target_sensor: DataFrame, threshold_corr: float=0.9) -> [bool, float]:
    """
    Check if the target sensor must be checked, because it has not reference with the reference sensor.
    Return [bool, float]: [{Is correlation ok?}, {Correlation percent}]

    corr = 0, No correlation
    corr = [-1, 0), Negative correlation
    corr = (0, 1], Positive correlation

    :params:
    :df_reference_sensor: DataFrame, Reference Tangara sensor
    :df_target_sensor: DataFrame, Target Tangara sensor
    :threshold_corr: float, Threshold to check the positive correlation percent between both Tangara sensors

    :return: [bool, float], There is not a correlation?
    """
    # Pearson Correlation Coefficient
    corr = 0
    warnings.simplefilter("ignore", category=NearConstantInputWarning)
    warnings.simplefilter("ignore", category=ConstantInputWarning)
    if (not df_reference_sensor.hasnans and not df_target_sensor.hasnans) and (df_reference_sensor.shape[0] == df_target_sensor.shape[0]):
        corr, _ = pearsonr(df_reference_sensor, df_target_sensor) if df_target_sensor.std() != 0 else PearsonRResult(0,0,alternative=0,n=0)
        corr = 0 if math.isnan(corr) else corr
        
    # corr = 0, No correlation
    # corr = [-1, 0), Negative correlation
    # corr = (0, 1], Positive correlation
    # Threshold
    if corr < threshold_corr:
        # There is not correlation
        return [False, float("{:.2f}".format(corr))]
    # There is correlation
    return [True, float("{:.2f}".format(corr))]

In [8]:
def df_to_csv(df: DataFrame, filename: str, datafolder: str='0_raw') -> None:
    """
    Save DataFrame into data folder as a CSV file.
    datafolder: str, choice ['0_raw', '1_clean', '2_features', 'backup']

    :params:
    :df: DataFrame, pandas DataFrame
    :filename: str, CSV file name with extension .csv
    :datafolder: str, choice ['0_raw', '1_clean', '2_features', 'backup']
    """
    # Save DataFrame into CSV file
    path_datafolder=f"{Path().resolve().parents[1]}/standalone/data/{datafolder}"
    df.to_csv(f"{path_datafolder}/{filename}")

In [9]:
def df_from_csv(filename: str, datafolder: str='0_raw', dtindex: bool=True) -> DataFrame:
    """
    Load DataFrame from CSV file localted in data folder.
    datafolder: str, choice ['0_raw', '1_clean', '2_features', 'backup']

    :params:
    :filename: str, CSV file name with extension .csv
    :datafolder: str, choice ['0_raw', '1_clean', '2_features', 'backup']
    :dtindex: bool, default True, Does the CSV file include and DATETIME column?
    
    :return: df: DataFrame, pandas DataFrame
    """
    # Load DataFrame from CSV file
    path_csvfile=f"{Path().resolve().parents[1]}/standalone/data/{datafolder}/{filename}"
    df_data = pd.read_csv(path_csvfile)
    if dtindex:
        df_data = df_data.set_index('DATETIME')
        df_data.index = pd.to_datetime(df_data.index)
        df_data = df_data.tz_convert("America/Bogota")
    return df_data

In [10]:
def df_tangara_sensors(start_timestamp: int, end_timestamp: int) -> DataFrame:
    """
    Get Data Frame Tangaras Sensors of all Tangara sensors that have reported data over a period of time.

    :params:
    :start_timestamp: int, timestamp datetime value, ms
    :end_timestamp: int, timestamp datetime value, ms

    :return: DataFrame, Tangaras Sensors
    """
    # Query Tangaras
    influxdb_sql_query_tangaras = query_tangaras(start_timestamp, end_timestamp)
    # print("influxdb_sql_query_tangaras:", influxdb_sql_query_tangaras)
    # InfluxDB API REST Request
    influxdb_request = request_influxdb(influxdb_sql_query_tangaras)
    # print("influxdb_request:", influxdb_request)
    # print("influxdb_request.text:", influxdb_request.text)

    # Data Frame Tangaras
    df_tangaras = pd.DataFrame([], columns=['ID','GEOHASH','MAC','GEOLOCATION','LATITUDE','LONGITUDE'])

    # For each time series
    for result in influxdb_request.json()['results']:
        # print("result:", result)
        if 'series' in result:
            for serie in result['series']:
                # Get the serie
                # print("serie:", serie)

                # DataFrame by statement_id and df_tangaras['ID']
                tags = serie["tags"]
                columns = serie["columns"]
                values = serie["values"]
                # print("tags:", tags)
                # print("columns:", columns)
                # print("values:", values)
                df_tangara = pd.DataFrame(values, columns=columns)

                # Remove/Add Columns
                df_tangara['MAC'] = tags['name']
                df_tangara['GEOLOCATION'] = df_tangara['geohash'].apply(lambda x: " ".join(str(value) for value in list(geohash2.decode_exactly(x)[0:2])))
                df_tangara['LATITUDE'] = df_tangara['GEOLOCATION'].apply(lambda x: x.split(' ')[0])
                df_tangara['LONGITUDE'] = df_tangara['GEOLOCATION'].apply(lambda x: x.split(' ')[1])
                df_tangara['ID'] = df_tangara['MAC'].apply(lambda x: f"TANGARA_{x[-4:]}")
                df_tangara.rename(columns={'geohash': 'GEOHASH'}, inplace=True)
                df_tangara = df_tangara[['ID','GEOHASH','MAC','GEOLOCATION','LATITUDE','LONGITUDE']]

                # print('df_tangara.head():', df_tangara.head())

                df_tangaras = pd.concat([df_tangaras, df_tangara], ignore_index=True)
    
    # Set Index
    df_tangaras.set_index('ID', inplace=True)
    
    return df_tangaras

In [11]:
def df_data_sensors(df_tangaras: DataFrame, start_timestamp: int, end_timestamp: int, measure: str='pm25', group_by_time: str='30s') -> DataFrame:
    """
    Get Measure Data Frame Sensors of all Tangara sensors that have reported data over a period of time.
    
    :params:
    :df_tangaras: DataFrame, Tangaras DataFrame
    :start_timestamp: int, timestamp datetime value, ms
    :end_timestamp: int, timestamp datetime value, ms
    :measure: str, choice ['pm25', 'tmp', 'hum']
    :group_by_time: str, choice ['30s', '1m', '1h']

    :return: DataFrame, Measure Data Frame Sensors
    """
    # Data Frame Sensors List
    df_sensors_list: [DataFrame] = []
    # SQL Query Data Sensors
    influxdb_sql_query_measure = query_measure(df_tangaras['MAC'].to_list(), start_timestamp, end_timestamp, measure, group_by_time)# print("influxdb_sql_query_measure:", influxdb_sql_query_measure)
    # print("influxdb_sql_query_measure:", influxdb_sql_query_measure)
    # InfluxDB API REST Request
    influxdb_request = request_influxdb(influxdb_sql_query_measure)
    # print("influxdb_request:", influxdb_request)
    # print("influxdb_request.text:", influxdb_request.text)

    # For each time series
    for result in influxdb_request.json()['results']:
        if 'series' in result:
            # Get the series and the statement_id
            series = result['series'][0]
            statement_id = result.get('statement_id')
            # print("series:", series)
            # print("statement_id:", statement_id)

            # DataFrame by statement_id and df_tangaras['ID']
            columns = ["DATETIME", df_tangaras['ID'].to_list()[statement_id]]
            values = series["values"]
            df_sensor = pd.DataFrame(values, columns=columns)
            # Set Index on DATETIME
            df_sensor.set_index('DATETIME', inplace=True)
            # print('df_sensor.head():', df_sensor.head())

            # Append to df_sensors_list
            df_sensors_list.append(df_sensor)

    # Join all df_sensors into a single DataFrame
    df_sensors = df_sensors_list[0].join(df_sensors_list[1:]).reset_index()

    # Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
    tz = timezone(timedelta(hours=-5))
    df_sensors['DATETIME'] = df_sensors['DATETIME'].apply(lambda x: datetime.fromtimestamp(int(x) / 1000, tz=tz).isoformat())
    df_sensors['DATETIME'] = pd.to_datetime(df_sensors['DATETIME'])

    # SET GROUP BY TIME
    GROUP_BY_TIME = {'30s': '30S', '1m': 'Min', '1h': 'H'}
    group_by_time = GROUP_BY_TIME[group_by_time]

    # Set Index
    df_sensors.set_index('DATETIME', inplace=True)
    df_sensors = df_sensors.asfreq(freq=group_by_time)
    df_sensors = df_sensors.tz_convert("America/Bogota")

    df_sensors[df_sensors.columns.to_list()] = df_sensors[df_sensors.columns.to_list()].astype('float64')
    
    return df_sensors

In [12]:
def plot_hist(df_data: DataFrame, num_v_charts: int=4) -> None:
    """
    Plot Histograms
    Each column fron df_data will be plotted as Histogramas in a subplots.

    :params:
    :df_data: DataFrame, each column in data frame will be plotted
    :num_v_charts: int, default 4, how many colums the subplots will have
    """
    # Canvas
    num_cols = len(df_data.columns)
    num_rows = math.ceil(len(df_data.columns)/num_v_charts)
    fig, axes = plt.subplots(num_rows, num_v_charts, figsize=(22, 16), tight_layout=True, sharey=True)
    fig.suptitle('Histograms', fontsize=16)
    # Plot
    k = 0
    for i in range(0, num_rows):
        for j in range(0, num_v_charts):
            if k < num_cols:
                sns.histplot(ax=axes[i, j], data=df_data[df_data.columns[k]], kde=True, discrete=False)
            k += 1

In [13]:
def plot_boxes(df_data: DataFrame, num_v_charts: int=4) -> None:
    """
    Plot Boxplots
    Each column fron df_data will be plotted as Boxplot in a subplots.

    :params:
    :df_data: DataFrame, each column in data frame will be plotted
    :num_v_charts: int, default 4, how many colums the subplots will have
    """
    # Canvas
    num_cols = len(df_data.columns)
    num_rows = math.ceil(len(df_data.columns)/num_v_charts)
    fig, axes = plt.subplots(num_rows, num_v_charts, figsize=(22, 16), tight_layout=True)
    fig.suptitle('Boxplots', fontsize=16)
    # Plot
    k = 0
    for i in range(0, num_rows):
        for j in range(0, num_v_charts):
            if k < num_cols:
                sns.boxplot(ax=axes[i, j], data=df_data[[df_data.columns[k]]], orient="h", x=df_data.columns[k])
            k += 1

In [14]:
def plot_lines(df_data: DataFrame, num_v_charts: int=4) -> None:
    """
    Plot Lineplots
    Each column fron df_data will be plotted as Lineplot in a subplots.

    :params:
    :df_data: DataFrame, each column in data frame will be plotted
    :num_v_charts: int, default 4, how many colums the subplots will have
    """
    # Canvas
    num_cols = len(df_data.columns)
    num_rows = math.ceil(len(df_data.columns)/num_v_charts)
    fig, axes = plt.subplots(num_rows, num_v_charts, figsize=(22, 16), tight_layout=True, sharey=False)
    fig.suptitle('Lineplots', fontsize=16)
    # Plot
    k = 0
    for i in range(0, num_rows):
        for j in range(0, num_v_charts):
            if k < num_cols:
                sns.lineplot(ax=axes[i, j], data=df_data[[df_data.columns[k]]], markers=True)
                axes[i, j].tick_params(axis='x', rotation=30)
                axes[i, j].set_ylabel("PM2.5")
                
                locator = mdates.AutoDateLocator(minticks=3, maxticks=7)
                formatter = mdates.ConciseDateFormatter(locator)
                
                axes[i, j].xaxis.set_major_locator(locator)
                axes[i, j].xaxis.set_major_formatter(formatter)

            k += 1

In [15]:
def drop_pm25_outliers(df_pm25_raw: DataFrame) -> [DataFrame, dict]:
    """
    Drop PM2.5 Outliers
    Apply mask to drop outliers from PM2.5 values,
    return a DataFrame without outliers and a resume dictionary by dataframe column

    :params:
    :df_pm25_raw: DataFrame, PM2.5 Raw Data Values

    :return: [DataFrame, dict], Clean DataFrame and resume dictionary
    """
    # Resume
    resume = {}
    # Each Data Sensor
    for column in df_pm25_raw.columns:
        # Apply mask to max value allowed, Max AQI Scale Value
        df_pm25_raw[column] = df_pm25_raw[column].mask(df_pm25_raw[column] > PM25_MAX_VALUE, PM25_MAX_VALUE)
        
        # Standard Deviation
        std = df_pm25_raw[column].std()
        # Range = max value - min value
        rango = df_pm25_raw[column].max() - df_pm25_raw[column].min()
        # Quantiles
        media = df_pm25_raw[column].median()
        Q1 = df_pm25_raw[column].quantile(q=0.25)
        Q2 = df_pm25_raw[column].quantile(q=0.50)
        Q3 = df_pm25_raw[column].quantile(q=0.75)
        min_val = df_pm25_raw[column].quantile(q=0)
        max_val = df_pm25_raw[column].quantile(q=1.0)
        # Interquartil Range
        iqr = Q3 - Q1
        # Limites para deteccion de outliers (solo aplica para datos simetricamente distribuidos)**
        min_limit = Q1 - 1.5 * iqr
        max_limit = Q3 + 1.5 * iqr
        # Medidas de Tendencia Central**
        resume[column] = {
            "std": std,
            "rango": rango,
            "media": media,
            "Q1": Q1,
            "Q2": Q2,
            "Q3": Q3,
            "min_val": min_val,
            "max_val": max_val,
            "iqr": iqr,
            "min_lim": min_limit,
            "max_lim": max_limit
        }

        # Drop Outliers, using max_limit to apply mask and drop outliers
        df_pm25_raw[column] = df_pm25_raw[column].mask(df_pm25_raw[column] > max_limit, None)
    
    return [df_pm25_raw, resume]

In [16]:
def df_pm25_to_df_aqi(df_pm25: DataFrame) -> DataFrame:
    """
    PM2.5 DataFrame Sensors to AQI DataFrame Sensors

    :params:
    :df_pm25: DataFrame, PM2.5 DataFrame Sensors
    
    :return: DataFrame, AQI DataFrame Sensors
    """
    aqi_values = df_pm25.copy()
    for column in aqi_values.columns:
        aqi_values[column] = aqi_values[column].apply(lambda x: pm25_to_aqi(x))
        aqi_values[column] = aqi_values[column].astype('Int64')
    return aqi_values